# Binomial Tree Model

The binomial tree model is an extension of the binomial branch model, with more discrete time steps over the life of a derivative.  More complexities are introduced not only with more layers in the framework, but also with different payoff functions of derivatives products enabled by such a framework: vanilla European call/put, American options, exotic options like European/American Lookback call/put, Asian options, etc.  Some of them are path-independent, like vanilla European call; others are path-dependent, such as Lookback options.

Despite the complexities of this framework and different payoff rules, the underlying principles are the same as the binomial branch model, centering around no-arbitrage axiom, and martingale with $Q$-measure.

Let's take a simple case to start with: pricing a 3-layer derivative and payoffs are 100 if stock price ends up higher than that at time 0, and 0 otherwise (Ct.).  Later, building on this model, we can develop a more general implementation of the framework.

Input
```
    1. A tree with realizations of stock prices at each time t in {0, 1, 2}
    2. Continuously compounded risk-free interest rate r
    3. Strike price K
```

In [1]:
import math

In [2]:
class Node:
    """Assume a complete binary tree"""
    def __init__(self, value=0, t=0, up=None, down=None, q=0):
        """
            Initialize a new node in a generic binomial tree.
        """
        self.value = value
        self.t = t
        self.up = up
        self.down = down
        self.q = q
        self.x = 0
    
    def calculate_value(self, r, dt):
        """
            Calculate, update and return the value of a node recursively under Q measure.
        """
        if self.up != None and self.down != None:
            q = self.q
            discount_factor = math.exp(-r * dt)
            self.value = discount_factor * (q * self.up.calculate_value(r, dt) + (1 - q) * self.down.calculate_value(r, dt))
        return self.value
    
    def calculate_q(self, r, dt):
        """
            Calculate the risk-neutral probabilities of a binomial stock tree.  
            Once the function is called, probabilities of nodes in the whole subtree 
            with current node as root will be updated.
        """
        if self.up != None and self.down != None:
            self.q = (self.value * math.exp(r * dt) - self.down.value) / (self.up.value - self.down.value)
            self.up.calculate_q(r, dt)
            self.down.calculate_q(r, dt)
        return self.q

In [3]:
# some setup for test case
s1 = Node(value=215)
s2 = Node(value=230, t=1)
s3 = Node(value=190, t=1)
s1.up = s2
s1.down = s3

s4 = Node(value=240, t=2)
s5 = Node(value=220, t=2)
s6 = Node(value=200, t=2)
s7 = Node(value=180, t=2)
s2.up = s4
s2.down = s5
s3.up = s6
s3.down = s7

In [4]:
s1.calculate_q(r=.05, dt=.25)

0.69260917703091

In [5]:
s1.q, s2.q, s3.q

(0.69260917703091, 0.6446521927172952, 0.6194952896360263)

In [6]:
# setup for calculating derivative contract value
f4 = Node(value=100, t=2)
f5 = Node(value=100, t=2)
f6 = Node(value=0, t=2)
f7 = Node(value=0, t=2)

f2 = Node(t=1, up=f4, down=f5, q=s2.q)
f3 = Node(t=1, up=f6, down=f7, q=s3.q)

f1 = Node(t=0, up=f2, down=f3, q=s1.q)

In [7]:
f1.calculate_value(r=.05, dt=.25)

67.55085955200326

In [8]:
f1.value, f2.value, f3.value, f4.value, f5.value, f6.value, f7.value

(67.55085955200326, 98.75778004938815, 0.0, 100, 100, 0, 0)

In [9]:
# calculate hedge ratio
x1 = (f2.value - f3.value) / (s2.value - s3.value)
x2 = (f4.value - f5.value) / (s4.value - s5.value)
x3 = (f6.value - f7.value) / (s6.value - s7.value)

In [10]:
x1, x2, x3

(2.4689445012347035, 0.0, 0.0)